# Trabajo Práctico Nº1



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la

**Ejericicio 1.**
Implementar un programa que reciba como input el tamaño $J$  y devuelva la matriz $\boldsymbol{A}$. (Sug.: Revisar los usos del comandos ```np.diag```).

In [ ]:
def armar_A(J):
  unos = np.ones(J)
  P    = np.diag(-2*unos)
  U    = np.diag(unos[1:],1)
  L    = np.diag(unos[1:],-1)
  A = P+U+L
  return A

**Ejercicio 2.** Implementar los programas de los ejercicios 13 y 16 de la Práctica 2. Verificar que funcionen correctamente en ejemplos pequeños.



*Ejercicio 13* Escribir funciones de Python que calculen la solucion de un sistema:
1. $U\boldsymbol{x} = \boldsymbol{b}$, siendo U triangular superior.
      
2. $L\boldsymbol{y} = \boldsymbol{x}$, siendo L triangular inferior.

In [ ]:
def tri_U(U,b):
    x= np.empty(b.shape)*0  # inicializo x = array[0, 0, 0, ...]
    for i in range(b.shape[0]-1,-1,-1):
        x[i]=( b[i]- (U[i,i+1:]@x[i+1:])) /U[i,i]
    return x

def tri_L(L,x):
    y= np.empty(x.shape)*0 # inicializo array
    for i in range(x.shape[0]):
        y[i]= (x[i]-(L[i,:i]@y[:i]))/L[i,i]
    return y

In [ ]:
U = np.triu(np.random.random((3,3)))
b = np.random.random(3)
x = tri_U(U,b)
U, U@x, b

(array([[0.58530985, 0.14732435, 0.14964013],
        [0.        , 0.36547117, 0.18319538],
        [0.        , 0.        , 0.48053077]]),
 array([0.76702819, 0.81916835, 0.49893408]),
 array([0.76702819, 0.81916835, 0.49893408]))

In [ ]:
L = np.tril(np.random.random((3,3)))
x = np.random.random(3)
y = tri_L(L,x)
L, L@y, x

(array([[0.66592029, 0.        , 0.        ],
        [0.03768677, 0.28196331, 0.        ],
        [0.50424689, 0.8045346 , 0.76866151]]),
 array([0.77857067, 0.5580949 , 0.90762021]),
 array([0.77857067, 0.5580949 , 0.90762021]))

*Ejercico 16* Escribir funciones de Python que realicen las siguientes tareas:
1. Calcular la descomposición $LU$ de una matriz dada $A$, asumiendo que no es necesario
realizar pivoteos.
2. Llamando a la anterior y a las del *ejercicio 13*, resolver un sistema $A \boldsymbol{x} = b$.


In [ ]:
def descomposicion_LU(A):

    U = A.copy()
    filas, columnas = A.shape
    L = np.eye(filas)
    for i in range(filas-1):
        # Supongo que no debo permutar
        pivote = U[i]

        for k in range(i+1, filas):
            # Proceso de triangulacion tomo en coef los a_[]/a_ii
            # luego voy agreganndo ceros debajo de la filas de mi pivote
            # para conseguir U. L es formado por los coef.
            coef  = U[k,i]/U[i,i]
            L[k][i] = coef
            U[k]  = U[k] - coef * pivote

    return L, U

In [ ]:
A = np.random.random((3,3))
L, U = descomposicion_LU(A)
print(A)
print(L)
print(U)
print(L@U)
print("Restamos para verficar A-LU=0 \n", np.round(A-L@U,2))

[[0.70956242 0.99972686 0.50570851]
 [0.61763757 0.69379021 0.0411498 ]
 [0.30741204 0.56214176 0.78876418]]
[[ 1.          0.          0.        ]
 [ 0.87044854  1.          0.        ]
 [ 0.43324171 -0.73131139  1.        ]]
[[ 0.70956242  0.99972686  0.50570851]
 [ 0.         -0.17642058 -0.39904343]
 [ 0.          0.          0.27784515]]
[[0.70956242 0.99972686 0.50570851]
 [0.61763757 0.69379021 0.0411498 ]
 [0.30741204 0.56214176 0.78876418]]
Restamos para verficar A-LU=0 
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [ ]:
def resolver_por_LU(A,b):

    L, U = descomposicion_LU(A)
    y    = tri_L(L,b)
    x    = tri_U(U,y)

    return x

In [ ]:
A_sistema = np.random.random((3,3))*10
b_sistema = np.round(np.random.random(3)*10)
x = resolver_por_LU(A,b)
print( "Resolvemos Ax=b \n")
print("A = ", A)
print("x = ", x)
print("b = ", b)
print("Ax-b=0 ", np.round(A@x-b))

Resolvemos Ax=b 

A =  [[-2.  1.  0.]
 [ 1. -2.  1.]
 [ 0.  1. -2.]]
x =  [-1.10958884 -1.45214948 -0.97554178]
b =  [0.76702819 0.81916835 0.49893408]
Ax-b=0  [-0.  0. -0.]


**Ejercicio 3.** Convencerse de que al aplicar el algoritmo LU a una matriz tridiagonal solo es necesario operar con los elementos de las tres diagonales no nulas y tanto L como U quedan bidiagonales.

In [ ]:
A = armar_A(3)
L, U = descomposicion_LU(A)
lista = [A, L, U, L@U]
for i in range(len(lista)):
  print(lista[i])

[[-2.  1.  0.]
 [ 1. -2.  1.]
 [ 0.  1. -2.]]
[[ 1.          0.          0.        ]
 [-0.5         1.          0.        ]
 [-0.         -0.66666667  1.        ]]
[[-2.          1.          0.        ]
 [ 0.         -1.5         1.        ]
 [ 0.          0.         -1.33333333]]
[[-2.  1.  0.]
 [ 1. -2.  1.]
 [ 0.  1. -2.]]
